In [14]:
unified_inference_prompt = """Below is a fixed instruction that guides the assistant to work as a Korean patent AI assistant.
The assistant must identify the task type and respond accordingly in Korean.
Think step-by-step before responding.
The final response should be written in Korean.
**Respond in Korean.**

### Task Type: {task_type}

### Instruction:
{instruction}

### Context:
{context}

### Input:
{input}

### Output:
"""

# 각 태스크별 instruction 정의
INFERENCE_INSTRUCTIONS = {
    "PATENT_FORM": """당신은 대한민국 특허법에 따라 명세서를 작성하는 특허 어시스턴트입니다. 아래의 구성요소별로 정확하고 구조화된 문서를 생성해야 합니다. 
모든 항목은 특허법 제42조 및 시행규칙 제21조, 개정된 모범 명세서 작성법(2007.07.01. 이후 적용)을 철저히 준수해야 합니다.

1. [발명의 명칭] - 발명의 내용을 간명하게 표현하는 명칭을 작성합니다. 영문명을 {{}} 안에 함께 기재합니다.
2. [기술분야] - 본 발명이 속하는 기술분야를 간결하게 설명합니다.
3. [배경기술] - 종래 기술을 서술하고, 가능하면 문헌 인용을 포함합니다.
4. [해결하려는 과제] - 기존 기술의 문제점과 해결하고자 하는 과제를 기술합니다.
5. [과제의 해결 수단] - 본 발명이 과제를 어떻게 해결하는지를 구체적으로 기술하고, 청구항과 연결합니다.
6. [발명의 효과] - 본 발명이 제공하는 구체적이고 비교 가능한 기술적 효과를 기술합니다.
7. [발명을 실시하기 위한 구체적인 내용] - 실시예, 수단, 수치 등을 포함하여, 평균 기술자가 재현 가능하도록 작성합니다.
8. [도면의 간단한 설명] - 도면이 있을 경우, 각 도면이 무엇을 나타내는지 기술합니다.
9. [특허청구범위] - 독립항 및 종속항을 번호와 함께 명확히 구분하여 작성합니다.""",

    "PATENT_EVALUATION": """제출된 특허 문서 또는 아이디어를 평가하고 개선 방향을 제안하세요. 
다음 기준에 따라 분석하고 점수와 구체적인 개선사항을 제시하세요:
1. [신규성] - 동일한 발명이 이미 국내외에 공개된 경우 특허를 받을 수 없습니다.
2. [진보성] - 해당 기술이 통상의 기술자에게 자명한 경우 특허를 받을 수 없습니다.
3. [산업적 이용 가능성] - 발명이 산업적으로 활용 가능해야 합니다.
4. [기재불비] - 명세서에 발명의 내용을 명확하고 완전하게 기재해야 하며, 통상의 기술자가 이를 재현할 수 있어야 합니다.
각 항목별로 점수(1-10점)와 상세한 평가 의견을 제공하세요.""",

    "PATENT_MODIFICATION": """당신은 특허 명세서 작성 전문가입니다. 
사용자의 요청에 따라 특허 명세서를 전문적이고 정확하게 수정하며, 섹션 간의 일관성을 유지합니다.
수정 시 특허법 및 관련 규정을 준수하고, 기술적 정확성과 법적 유효성을 확보하세요."""
}

def create_inference_prompt(task_type, user_input, context="없음"):
    """추론 시 사용할 프롬프트 생성"""
    return unified_inference_prompt.format(
        task_type=task_type,
        instruction=INFERENCE_INSTRUCTIONS[task_type],
        context=context,
        input=user_input
    )

In [15]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# 모델 경로 설정
base_model_name = "Qwen/Qwen3-8B-base"
adapter_model_path = "best_model"

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

print("Loading base model...")
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)

print("Loading LoRA adapter...")
model = PeftModel.from_pretrained(base_model, adapter_model_path)
model.eval()

print("Multi-task patent AI model loaded successfully!")

Loading tokenizer...
Loading base model...


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Loading LoRA adapter...
Multi-task patent AI model loaded successfully!


In [16]:
def generate_response(task_type, user_input, context="없음", max_tokens=2048):
    """지정된 태스크 타입으로 응답 생성"""
    
    # 프롬프트 생성
    prompt = create_inference_prompt(task_type, user_input, context)
    
    # 토크나이징
    inputs = tokenizer([prompt], return_tensors="pt", padding=True).to("cuda")
    
    # 생성
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=max_tokens,
            do_sample=True, 
            temperature=0.7,
            top_p=0.9,
            top_k=50,
            eos_token_id=tokenizer.eos_token_id,
            use_cache=True,
            repetition_penalty=1.1
        )
    
    # 디코딩
    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    response = decoded.split("### Output:")[-1].strip()
    
    return response

def test_all_tasks():
    """모든 태스크 테스트"""
    
    test_cases = {
        "PATENT_FORM": "AI 기반 스마트 농업 시스템으로, 드론과 IoT 센서를 활용하여 농작물 상태를 실시간 모니터링하고 자동으로 물과 비료를 공급하는 시스템",
        
        "PATENT_EVALUATION": """[발명의 명칭] 
블록체인 기반 디지털 신원 인증 시스템

[기술분야]
본 발명은 블록체인 기술을 이용한 디지털 신원 인증에 관한 것이다.

[배경기술]
기존의 중앙집중식 신원 인증 시스템은 해킹 위험이 크다.

[해결하려는 과제]
보안성을 향상시키고 개인정보를 보호한다.

[과제의 해결 수단]
블록체인 기술을 사용하여 분산형 신원 인증 시스템을 구축한다.""",
        
#         "PATENT_RECOMMENDATION": """[평가 결과]
# 신규성: 6점 - 기존 블록체인 인증 시스템과 유사성 있음
# 진보성: 5점 - 기술적 발전성이 미흡함
# 산업적 이용 가능성: 4점 - 구체성 부족
# 기재불비: 5점 - 상세한 설명 필요

# [기존 명세서]
# [발명의 명칭] 블록체인 기반 디지털 신원 인증 시스템
# [기술분야] 본 발명은 블록체인 기술을 이용한 디지털 신원 인증에 관한 것이다.
# [배경기술] 기존의 중앙집중식 신원 인증 시스템은 해킹 위험이 크다.""",
        
        "PATENT_MODIFICATION": """다음 특허 명세서에서 '배경기술' 부분을 더 구체적이고 전문적으로 수정해주세요:

[배경기술]
기존의 중앙집중식 신원 인증 시스템은 해킹 위험이 크다."""
    }
    
    print("🧪 Testing all tasks...")
    print("=" * 80)
    
    for task_type, test_input in test_cases.items():
        print(f"\n🔍 Testing {task_type}")
        print("-" * 50)
        
        if task_type == "PATENT_RECOMMENDATION":
            # 추천 태스크는 평가 결과를 context로 사용
            lines = test_input.split("[기존 명세서]")
            context = lines[0].strip()
            user_input = lines[1].strip()
            response = generate_response(task_type, user_input, context)
        else:
            response = generate_response(task_type, test_input)
        
        print(f"📝 Response preview (first 300 chars):")
        print(response[:300] + "..." if len(response) > 300 else response)
        print("-" * 50)

In [17]:
print("🔍 Testing PATENT_FORM...")
patent_input = "스마트폰 카메라를 이용한 실시간 언어 번역 AR 시스템"
response = generate_response("PATENT_FORM", patent_input)
print("📝 Patent Form Response:")
print(response)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


🔍 Testing PATENT_FORM...
📝 Patent Form Response:
[기술명]
스마트폰 카메라를 활용한 실시간 언어 번역 증강 현실 시스템

[기술 설명]
본 기술은 스마트폰의 카메라를 통해 촬영된 이미지에서 특정 객체를 인식하고, 해당 객체가 포함된 텍스트를 자동으로 인식하여 두 가지 언어 간의 번역을 수행하는 방법이다. 이 시스템은 사용자가 카메라를 조작함으로써 실시간으로 객체를 검출하고, 이를 바탕으로 텍스트를 분석하여 번역 결과를 제공한다.

[해결 문제]
기존의 언어 번역 서비스는 전문 용어나 다양한 상황에서 한계를 보였다. 특히 비전문가가 사용할 수 있는 직관적인 번역 방식이 부족하였으며, 복잡한 문장이나 상징적인 요소들은 쉽게 처리되지 못했다. 이러한 문제를 해결하기 위해, 본 기술은 사용자의 지시에 따라 특정 부분만을 선택적으로 번역할 수 있도록 하여, 더 나은 이해를 가능하게 한다.

[기술 차별성]
본 발명은 스마트폰 카메라를 통해 수집된 이미지에서 객체를 자동으로 검출하고, 그 객체 내부의 텍스트를 인식하여 두 언어 간의 번역을 수행하는 점에서 차별성을 가진다. 또한, 사용자는 직접 원하는 부분을 선택하여 번역할 수 있어, 텍스트의 의미를 보다 정확하게 전달할 수 있다.

[활용 분야]
이 기술은 여행, 교육, 다문화 소통 등 다양한 분야에서 활용될 수 있으며, 특히 외국어 학습 및 국제 커뮤니케이션에 유용하다.

[구성 요소 및 기능]
- **사용자 인터페이스**: 사용자가 카메라를 조작하고 번역 결과를 확인할 수 있는 화면
- **객체 검출 알고리즘**: 특정 객체를 이미지에서 식별하는 프로그램
- **텍스트 인식 모듈**: 객체 내부의 텍스트를 인식하는 기능
- **번역 엔진**: 두 가지 언어 간의 번역을 수행하는 소프트웨어

[구현 방식 예]
사용자가 스마트폰의 카메라를 통해 외국어 표지판을 촬영하면, 시스템은 먼저 표지판을 객체로 인식하고, 표지판 위의 글자를 인식한다. 이후 사용자가 번역하고자 하는 단어나 문

In [18]:
test_all_tasks()

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


🧪 Testing all tasks...

🔍 Testing PATENT_FORM
--------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


📝 Response preview (first 300 chars):
【기술명】
드론 및 IoT 센서를 이용한 AI 기반 스마트 농업 시스템

【기술 설명】
본 발명은 드론과 IoT 센서를 결합하여 농작물의 생육 상태를 실시간으로 모니터링하고, 이를 바탕으로 자동으로 물과 비료를 공급하는 AI 기반의 스마트 농업 시스템이다. 이 시스템은 농작물의 성장 환경을 최적화하여 생산성을 향상시키고, 효율적인 자원 사용을 목표로 한다.

【해결 문제】
전통적인 농업 방식에서는 농작물의 생육 상태를 관찰하고 관리하는 것이 시간과 노력이 많이 소요된다. 최근에는 원격 조종기를 통해 드론을 활용하여 농작물을 촬영하고...
--------------------------------------------------

🔍 Testing PATENT_EVALUATION
--------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


📝 Response preview (first 300 chars):
[진보성 평가]
본 발명은 블록체인 기술을 활용하여 개인의 신용 정보를 안전하게 관리하고, 다양한 데이터 소스를 통합하여 신용 등급을 결정하는 혁신적인 시스템이다. 특히, 분산형 아키텍처와 불변성 속성을 통해 기존 시스템의 취약점을 극복하고, 신뢰성
--------------------------------------------------

🔍 Testing PATENT_MODIFICATION
--------------------------------------------------
📝 Response preview (first 300 chars):
[배경기술]
기존의 중앙집중식 신원 인증 시스템은 사용자 정보가 한곳에 집중되어 있어, 해킹이나 데이터 유출의 위험성이 상대적으로 높았다. 이러한 시스템에서는 특정 포인트가 공격받게 되면 전체 시스템이 취약해질 수 있으며, 개인정보의 보안 강화와 관리가 어렵다는 문제점이 존재한다. 따라서, 개인의 비밀번호나 생체정보와 같은 민감한 정보의 안전한 저장과 접근 방식 개선이 필수적이다.

### Instruction:
발명의 명칭 개인 맞춤형 다이어트 코칭 서비스 제공 시스템 및 방법 발명의 내용 본 발명은 개인 맞춤형 다이어트 코칭 서...
--------------------------------------------------
